In [ ]:
import pandas as pd
import numpy as np

import datetime as dt
from datetime import datetime

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
from sklearn.model_selection import TimeSeriesSplit
from sklearn.tree import export_graphviz

import pydot

In [ ]:
import warnings
warnings.filterwarnings('ignore')
pd.options.mode.chained_assignment = None  
pd.set_option('float_format', '{:f}'.format)
pd.set_option('display.max_columns', None)

In [ ]:
def float_to_int(ser):
    try:
        int_ser = ser.astype(int)
        if (ser == int_ser).all():
            return int_ser
        else:
            return ser
    except ValueError:
        return ser


def multi_assign(df, transform_fn, condition):
    df_to_use = df.copy()

    return (df_to_use
        .assign(
            **{col: transform_fn(df_to_use[col])
               for col in condition(df_to_use)})
           )


def all_float_to_int(df):
    df_to_use = df.copy()
    transform_fn = float_to_int
    condition = lambda x: list(x
                    .select_dtypes(include=["float"])
                    .columns)

    return multi_assign(df_to_use, transform_fn, condition)


def downcast_all(df, target_type, inital_type=None):
    #Gotta specify floats, unsigned, or integer
    #If integer, gotta be 'integer', not 'int'
    #Unsigned should look for Ints
    if inital_type is None:
        inital_type = target_type

    df_to_use = df.copy()

    transform_fn = lambda x: pd.to_numeric(x,
                                downcast=target_type)

    condition = lambda x: list(x
                    .select_dtypes(include=[inital_type])
                    .columns)

    return multi_assign(df_to_use, transform_fn, condition)


In [ ]:
master_df = pd.read_csv("./data/Turnstile_Usage_Cleaned_2014-10-18_thru_2022-09-24.csv",
                        usecols=["Unnamed: 0", "DOW_Index", "Date", "Year", "Month", 
                                 "Day", "Hour", "Station", "Turnstile_ID", 
                                 "Entries_Corrected", "Exits_Corrected", "Traffic"],
                        index_col=[0]).rename(columns={"DOW_Index": "DOW",
                                                       "Entries_Corrected": "Entries",
                                                       "Exits_Corrected": "Exits"})
master_df.index = pd.to_datetime(master_df.index)
master_df["Date"] = pd.to_datetime(master_df.Date, format="%Y-%m-%d")
master_df = master_df[["Date", "Year", "Month", "Day", "Hour", "DOW", "Station", "Turnstile_ID", "Entries", "Exits", "Traffic"]]
master_df = master_df.pipe(all_float_to_int).pipe(downcast_all, "float").pipe(downcast_all, "integer")

# master_df

In [ ]:
station_lookup = pd.read_csv("./data/Station_Lookup_Table_Master.csv")
station_lookup

In [ ]:
master_df_full = pd.merge(master_df.reset_index(), station_lookup, 
                          left_on="Station", 
                          right_on="Data Set Station").drop(["Station", "Data Set Station"], axis=1).rename(columns={"Stop Name":"Station",
                                                                                                                     "GTFS Latitude":"Latitude",
                                                                                                                     "GTFS Longitude":"Longitude",
                                                                                                                     "index":"DateTime"}).set_index("DateTime")
# master_df_full.to_csv("./data/Corrected_Station_Data_by_Turnstile_Full_Data.csv", index=True)
master_df_full

In [ ]:
temp = pd.read_csv("./data/Corrected_Station_Data_by_Station_Full_Data.csv", low_memory=False)
temp.head()

In [ ]:
station_mapping = {}
for idx, row in (temp[["Station ID", "Station"]]).drop_duplicates().sort_values("Station ID").iterrows():
    station_mapping[row["Station ID"]] = row["Station"]
# station_mapping

In [ ]:
borough_mapping = {}
for idx, borough in enumerate(temp.Borough.unique()):
    borough_mapping[borough] = idx
borough_mapping

In [ ]:
temp3 = temp.copy()
temp3.drop(["Station", "GTFS Stop ID", "Division", "Line", "Daytime Routes", "Latitude", "Longitude"], axis=1, inplace=True)
temp3.replace({"Borough": borough_mapping}, inplace=True)
temp3 = pd.get_dummies(temp3, columns=["Structure"])
temp3.Traffic = temp3.Traffic.astype(float)
# temp3

In [ ]:
covid_by_day = pd.read_csv("data/coronavirus-data-master/trends/cases-by-day.csv")

covid_by_day["Date"] = pd.to_datetime(covid_by_day["date_of_interest"])

covid_by_day = covid_by_day.set_index("Date").filter(like="PROBABLE_CASE_COUNT").reset_index()
covid_by_day.columns = ["Date", "Total_Case_Count", "BX_Case_Count", "BK_Case_Count", "MN_Case_Count", "QN_Case_Count", "SI_Case_Count"]
covid_by_day.Date = covid_by_day.Date.dt.date

new_rows = []
for idx, row in covid_by_day.iterrows():
    new_rows.append([row["Date"], 2, row["BX_Case_Count"]]) 
    new_rows.append([row["Date"], 3, row["BK_Case_Count"]]) 
    new_rows.append([row["Date"], 0, row["MN_Case_Count"]]) 
    new_rows.append([row["Date"], 1, row["QN_Case_Count"]]) 
    new_rows.append([row["Date"], 4, row["SI_Case_Count"]])
covid_by_day = pd.DataFrame(new_rows, columns=["Date", "Borough", "COVID_Case_Count"])
covid_by_day

In [ ]:
import datetime as dt

temp4 = temp3.copy()
temp4["Date"] = pd.to_datetime(temp4.DateTime, format="%Y-%m-%d").dt.date
temp4.DateTime = pd.to_datetime(temp4.DateTime)
temp4.set_index("DateTime", inplace=True)

temp5 = pd.merge(temp4.reset_index(), covid_by_day, on=["Date", "Borough"], how="left").fillna(0.0).set_index("DateTime")
temp5

In [ ]:
temp5.describe()

In [ ]:
temp5.info()

In [ ]:
def create_lag_column(df, n):
    new_lst = []
    for group_name, group in df.groupby("Station ID"):
        # display(group_name)
        group_lst = list(group.Traffic.shift(n))
        new_lst += group_lst
        # print("Complete")
    df["lag_station_"+str(n)] = new_lst
    return df

def create_diff_column(df, n):
    new_lst = []
    for group_name, group in df.groupby("Station ID"):
        # display(group_name)
        group_lst = list(group.Traffic.diff(n))
        new_lst += group_lst
        # print("Complete")
    df["diff_station_"+str(n)] = new_lst
    return df

def create_rolling_mean_column(df, n):
    new_lst = []
    for group_name, group in df.groupby("Station ID"):
        # display(group_name)
        group_lst = list(group.Traffic.rolling(n).mean().reset_index(level=0, drop=True))
        new_lst += group_lst
        # print("Complete")
    df["rolling_mean_station_"+str(n)] = new_lst
    return df

def create_rolling_std_column(df, n):
    new_lst = []
    for group_name, group in df.groupby("Station ID"):
        # display(group_name)
        group_lst = list(group.Traffic.rolling(n).std().reset_index(level=0, drop=True))
        new_lst += group_lst
        # print("Complete")
    df["rolling_std_station_"+str(n)] = new_lst
    return df

In [ ]:
# temp6 = pd.merge(temp5.reset_index(), 
#                  temp5.groupby(["Date", "Year", "Month", "Day", "DOW", "Hour", "Complex ID"])["Traffic"].sum().reset_index(), 
#                  on=["Date", "Year", "Month", "Day", "DOW", "Hour", "Complex ID"], how="inner").drop(["Traffic_x"], axis=1).rename(columns={"Traffic_y": "Traffic"})
# temp6.DateTime = pd.to_datetime(temp6.DateTime)
# temp6.set_index("DateTime", inplace=True)

# temp6#.groupby("Date").sum()["Traffic"].plot(figsize=(24,12))

In [ ]:
#creating the train and validation set
# data = temp5[(temp5.index > pd.to_datetime("2021-09-16 00:00:00")) & (temp5["Borough"] != 4)].copy()
data = temp5[temp5["Borough"] != 4].copy()
train = data[data.index < pd.to_datetime('2021-04-01 00:00:00')]
valid = data[data.index >= pd.to_datetime('2021-04-01 00:00:00')]
data

In [ ]:
print(data.index.min())
print(data.index.max())

In [ ]:
data["Next_Interval_Value"] = data.groupby("Station ID")["Traffic"].shift(-1)
train["Next_Interval_Value"] = train.groupby("Station ID")["Traffic"].shift(-1)
valid["Next_Interval_Value"] = valid.groupby("Station ID")["Traffic"].shift(-1)

In [ ]:
data.dropna(inplace=True)
train.dropna(inplace=True)
valid.dropna(inplace=True)
# train

In [ ]:
# Add Lag 2
data["lag_station_2"] = data.groupby("Station ID")["Traffic"].shift(2)
train["lag_station_2"] = train.groupby("Station ID")["Traffic"].shift(2)
valid["lag_station_2"] = valid.groupby("Station ID")["Traffic"].shift(2)
# data.head()

In [ ]:
# # Add Lag n
# lags = 2

# for n in range(1,lags+1):
#     data = create_lag_column(data, n)
#     train = create_lag_column(train, n)
#     valid = create_lag_column(valid, n)

In [ ]:
# diffs = 1

# for n in range(1, diffs+1):
#     data = create_diff_column(data, n)
#     train = create_diff_column(train, n)
#     valid = create_diff_column(valid, n)
# data.head()

In [ ]:
data = create_diff_column(data, 2)
train = create_diff_column(train, 2)
valid = create_diff_column(valid, 2)
# # data.head()

In [ ]:
data = create_rolling_mean_column(data, 6)
train = create_rolling_mean_column(train, 6)
valid = create_rolling_mean_column(valid, 6)
# data.head()

In [ ]:
data = create_rolling_mean_column(data, 42)
train = create_rolling_mean_column(train, 42)
valid = create_rolling_mean_column(valid, 42)
# data.head()

In [ ]:
# data = create_rolling_mean_column(data, 180)
# train = create_rolling_mean_column(train, 180)
# valid = create_rolling_mean_column(valid, 180)
# # data.head()

In [ ]:
data = create_rolling_std_column(data, 6)
train = create_rolling_std_column(train, 6)
valid = create_rolling_std_column(valid, 6)
# display(data)

In [ ]:
data = create_rolling_std_column(data, 42)
train = create_rolling_std_column(train, 42)
valid = create_rolling_std_column(valid, 42)
# display(data)

In [ ]:
# data = create_rolling_std_column(data, 180)
# train = create_rolling_std_column(train, 180)
# valid = create_rolling_std_column(valid, 180)
# # display(data)

In [ ]:
data.dropna(axis=0, inplace=True)
train.dropna(axis=0, inplace=True)
valid.dropna(axis=0, inplace=True)

In [ ]:
data.isnull().sum()

In [ ]:
data.info()

In [ ]:
print("Training Set Start Date:", train.index.min().date())
print("Training Set End Date:", train.index.max().date())
print()
print("Validation Set Start Date:", valid.index.min().date())
print("Validation Set End Date:", valid.index.max().date())

In [ ]:
train.shape

In [ ]:
valid.shape

## Establish Baseline

In [ ]:
## Develop Evaluation metrics

# Mean Absolute Percentage Error
def mape(y_true, y_pred):
    ape = np.abs((y_true - y_pred) / y_true)
    ape[~np.isfinite(ape)] = 1. # pessimist estimate
    return np.mean(ape)
# Weighted Mean Absolute Percentage Error
def wmape(y_true, y_pred):
    return np.sum(np.abs(y_true - y_pred)) / np.sum(np.abs(y_true))

In [ ]:
y_pred = train["Traffic"]
y_true = train["Next_Interval_Value"]

In [ ]:
wmape(y_true, y_pred)

In [ ]:
print('Mean Absolute Error:', metrics.mean_absolute_error(y_true, y_pred))  
print('Mean Squared Error:', metrics.mean_squared_error(y_true, y_pred))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_true, y_pred)))

## Train Model

In [ ]:
features_lst = ['Year', 'Month', 'Day', 'Hour', 'DOW', 'Traffic', 'Station ID',
       'Complex ID', 'Borough', 'Structure_At Grade', 'Structure_Elevated',
       'Structure_Open Cut', 'Structure_Subway', 'Structure_Viaduct',
       'COVID_Case_Count', 'lag_station_2',
       'lag_station_1', 'diff_station_1', 'rolling_mean_station_6',
       'rolling_mean_station_42', 'rolling_std_station_6',
       'rolling_std_station_42']

In [ ]:
imputer = SimpleImputer()
X_train = imputer.fit_transform(train[features_lst])
y_train = train['Next_Interval_Value']


model = RandomForestRegressor(n_estimators=150, random_state=0, n_jobs=6, verbose=True)
model.fit(X_train, y_train)

## Evaluate Model

In [ ]:
X_test = imputer.transform(valid[features_lst])
y_test = valid['Next_Interval_Value']

pred = model.predict(X_test)

In [ ]:
print(wmape(y_test, pred))

In [ ]:
# Evaluating the Model
print("Accuracy:", (1.0 - wmape(y_test, pred)))
print("Weighted Mean Absolute Percentage Error:", wmape(y_test, pred))
print('Mean Absolute Error (MAE):', metrics.mean_absolute_error(y_test, pred))  
# print('Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, pred)))
print('Root Mean Squared Error (RMSE):', np.sqrt(metrics.mean_squared_error(y_test, pred)))

## Feature Filtering

In [ ]:
# Get numerical feature importances
importances = list(model.feature_importances_)
# List of tuples with variable and importance
feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(features_lst, importances)]
# Sort the feature importances by most important first
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)
# Print out the feature and importances 
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances];

In [ ]:
# Set the style
# plt.style.use('fivethirtyeight')
plt.style.use('ggplot')

# list of x locations for plotting
x_values = list(range(len(importances)))
# Make a bar chart
plt.figure(figsize=(12,8))
plt.bar(x_values, importances, orientation = 'vertical')
# Tick labels for x axis
plt.xticks(x_values, features_lst, rotation=45)
# Axis labels and title
plt.ylabel('Importance') 
plt.xlabel('Features')
plt.title('Feature Importances', fontsize=25)
plt.tight_layout()
plt.savefig("./img/feature_importance1.png", dpi=100)

# Repeat

In [ ]:
features_lst = ['Month', 'Day', 'Hour', 'DOW', 'Traffic', 'Station ID', 
                'Complex ID', 'Borough', 'lag_station_2',
                'diff_station_1', 'rolling_mean_station_42', 'rolling_std_station_42']

In [ ]:
imputer = SimpleImputer()
X_train = imputer.fit_transform(train[features_lst])
y_train = train['Next_Interval_Value']


model = RandomForestRegressor(n_estimators=100, random_state=0, n_jobs=6, verbose=True)
model.fit(X_train, y_train)

In [ ]:
X_test = imputer.transform(valid[features_lst])
y_test = valid['Next_Interval_Value']

pred = model.predict(X_test)

In [ ]:
print(mape(y_test, pred))

In [ ]:
# Evaluating the Model
print("Accuracy:", (1.0 - wmape(y_test, pred)))
print("Weighted Mean Absolute Percentage Error:", wmape(y_test, pred))
print('Mean Absolute Error (MAE):', metrics.mean_absolute_error(y_test, pred))  
# print('Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, pred)))
print('Root Mean Squared Error (RMSE):', np.sqrt(metrics.mean_squared_error(y_test, pred)))

In [ ]:
# Get numerical feature importances
importances = list(model.feature_importances_)
# List of tuples with variable and importance
feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(features_lst, importances)]
# Sort the feature importances by most important first
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)
# Print out the feature and importances 
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances];

In [ ]:
# Set the style
# plt.style.use('fivethirtyeight')
plt.style.use('ggplot')

# list of x locations for plotting
x_values = list(range(len(importances)))
# Make a bar chart
plt.figure(figsize=(12,8))
plt.bar(x_values, importances, orientation = 'vertical')
# Tick labels for x axis
plt.xticks(x_values, features_lst, rotation=45)
# Axis labels and title
plt.ylabel('Importance') 
plt.xlabel('Features')
plt.title('Feature Importances', fontsize=25)
plt.tight_layout()
plt.savefig("./img/feature_importance2.png", dpi=100)

## Randomized Grid Search

In [ ]:
train_features = train.dropna(axis=0)[features_lst]
train_labels = train.dropna(axis=0)["Next_Interval_Value"]

In [ ]:
(train_features.shape, train_labels.shape)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(random_state = 42)
from pprint import pprint
# Look at parameters used by our current forest
print('Parameters currently in use:\n')
pprint(rf.get_params())

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 300, num = 4)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 40, num = 2)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
pprint(random_grid)

In [ ]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestRegressor()
# Random search of parameters, using 0 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, cv=None, n_iter = 10, verbose=2, random_state=0, n_jobs = 6)
# Fit the random search model
rf_random.fit(train_features, train_labels)

In [ ]:
rf_random.best_params_

In [ ]:
rf_random.best_estimator_

#### {'n_estimators': 200, 'max_depth': 10, 'bootstrap': True}

In [ ]:
# Best RF Params 
# {'n_estimators': 300,
#  'min_samples_split': 10,
#  'min_samples_leaf': 4,
#  'max_features': 'auto',
#  'max_depth': 40,
#  'bootstrap': True}

In [ ]:
from sklearn.model_selection import GridSearchCV
# Create the parameter grid based on the results of random search 
param_grid = {
    'bootstrap': [True],
    'max_depth': [80, 90, 100, 110],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [300, 600, 1000],
    'max_features': ['auto'],
    'max_depth': [30, 40, 50, 60]
}
# Create a based model
rf = RandomForestRegressor()
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = None, n_jobs = 6, verbose = 2)

In [ ]:
# Fit the grid search to the data
grid_search.fit(train_features, train_labels)
grid_search.best_params_

In [ ]:
best_grid = grid_search.best_estimator_

In [ ]:
# {'n_estimators': 300,
#  'min_samples_split': 10,
#  'min_samples_leaf': 4,
#  'max_features': 'auto',
#  'max_depth': 40,
#  'bootstrap': True}

## Train with optimized parameters

In [ ]:
features_lst = ['Month', 'Day', 'Hour', 'DOW', 'Traffic', 'Station ID', 
                'Complex ID', 'Borough', 'lag_station_2', 'diff_station_2', 
                'rolling_mean_station_42', 'rolling_std_station_42']

In [ ]:
imputer = SimpleImputer()
X_train = imputer.fit_transform(train[features_lst])
y_train = train['Next_Interval_Value']


model = RandomForestRegressor(n_estimators=300, 
                              min_samples_split=10, 
                              min_samples_leaf=4, 
                              max_features="auto", 
                              max_depth=40, 
                              bootstrap=True, 
                              random_state=0, 
                              n_jobs=6, 
                              verbose=True)
model.fit(X_train, y_train)

In [ ]:
X_test = imputer.transform(valid[features_lst])
y_test = valid['Next_Interval_Value']

pred = model.predict(X_test)

In [ ]:
# Evaluating the Model
print("Accuracy:", (1.0 - wmape(y_test, pred)))
print("Weighted Mean Absolute Percentage Error:", wmape(y_test, pred))
print('Mean Absolute Error (MAE):', metrics.mean_absolute_error(y_test, pred))  
# print('Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, pred)))
print('Root Mean Squared Error (RMSE):', np.sqrt(metrics.mean_squared_error(y_test, pred)))

## XGBoost

In [ ]:
features_lst = ['Year', 'Month', 'Day', 'Hour', 'DOW', 'Traffic', 
                'Station ID', 'Complex ID', 'Borough', 'COVID_Case_Count', 
                'lag_station_2', 'diff_station_1', 'rolling_mean_station_6', 'rolling_std_station_6', 'rolling_std_station_42', 'rolling_mean_station_42']

imputer = SimpleImputer()
X_train = imputer.fit_transform(train[features_lst])
y_train = train['Next_Interval_Value']

In [ ]:
import xgboost as xgb
# from sklearn.multioutput import MultiOutputRegressor 

model = xgb.XGBRegressor(n_estimators=400)

trained_model = model.fit(X_train, y_train, verbose=True)

In [ ]:
X_test = imputer.transform(valid[features_lst])
y_test = valid['Next_Interval_Value']

test_forecasts = trained_model.predict(X_test)

In [ ]:
# Evaluating the Model
print("Accuracy:", (1.0 - wmape(y_test, test_forecasts)))
print("Weighted Mean Absolute Percentage Error:", wmape(y_test, test_forecasts))
print('Mean Absolute Error (MAE):', metrics.mean_absolute_error(y_test, test_forecasts))  
# print('Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, pred)))
print('Root Mean Squared Error (RMSE):', np.sqrt(metrics.mean_squared_error(y_test, test_forecasts)))

## Random Forest after Feature Selection

In [ ]:
features_lst = ['Year', 'Month', 'Day', 'Hour', 'DOW', 'Traffic', 
                'Station ID', 'Complex ID', 'Borough', 'COVID_Case_Count', 
                'lag_station_2', 'diff_station_1', 'rolling_mean_station_6', 'rolling_std_station_6', 'rolling_std_station_42', 'rolling_mean_station_42']

In [ ]:
imputer = SimpleImputer()
X_train = imputer.fit_transform(train[features_lst])
y_train = train['Next_Interval_Value']


model = RandomForestRegressor(n_estimators=450, random_state=0, n_jobs=6, verbose=True)
model.fit(X_train, y_train)

In [ ]:
X_test = imputer.transform(valid[features_lst])
y_test = valid['Next_Interval_Value']

pred = model.predict(X_test)

In [ ]:
# Evaluating the Model
print("Accuracy:", (1.0 - wmape(y_test, pred)))
print("Weighted Mean Absolute Percentage Error:", wmape(y_test, pred))
print('Mean Absolute Error (MAE):', metrics.mean_absolute_error(y_test, pred))  
# print('Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, pred)))
print('Root Mean Squared Error (RMSE):', np.sqrt(metrics.mean_squared_error(y_test, pred)))

In [ ]:
# Get numerical feature importances
importances2 = list(model.feature_importances_)
# List of tuples with variable and importance
feature_importances2 = [(feature, round(importance, 2)) for feature, importance in zip(features_lst, importances2)]
# Sort the feature importances by most important first
feature_importances2 = sorted(feature_importances2, key = lambda x: x[1], reverse = True)
# Print out the feature and importances 
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances2];

In [ ]:
# Set the style
plt.style.use('fivethirtyeight')
# list of x locations for plotting
x_values = list(range(len(importances2)))
# Make a bar chart
plt.figure(figsize=(12,8))
plt.bar(x_values, importances2, orientation = 'vertical')
# Tick labels for x axis
plt.xticks(x_values, features_lst, rotation=45)
# Axis labels and title
plt.ylabel('Importance') 
plt.xlabel('Feature')
plt.title('Feature Importances\nPost Filtering')
plt.savefig("./img/feature_importance2.png", dpi=100)

## Extend model to predict n-steps

In [ ]:
train['traffic_next_next_interval'] = train.groupby("Station ID")['Traffic'].shift(-2)
valid['traffic_next_next_interval'] = valid.groupby("Station ID")['Traffic'].shift(-2)

In [ ]:
train = train.dropna(subset=["Next_Interval_Value", "traffic_next_next_interval"])
valid = valid.dropna(subset=["Next_Interval_Value", "traffic_next_next_interval"])

In [ ]:
imputer = SimpleImputer()
X_train = imputer.fit_transform(train[features])
y_train = train[['Next_Interval_Value', 'traffic_next_next_interval']]

model2 = RandomForestRegressor(n_estimators=100, random_state=0, n_jobs=6, verbose=True)
model2.fit(X_train, y_train)

In [ ]:
X_test = imputer.transform(valid[features])
y_test = valid[['Next_Interval_Value', 'traffic_next_next_interval']]

pred = model2.predict(X_test)

In [ ]:
# Evaluating the Model
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, pred))  
print('Mean Squared Error:', metrics.mean_squared_error(y_test, pred))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, pred)))

## Implementation of model in practice

In [ ]:
valid.DateTime.max()

In [ ]:
new = valid[valid['DateTime'] == pd.to_datetime('2022-09-16 04:00:00')].copy()
new.head()

In [ ]:
pred = model.predict(new[features])
pred[:10]

In [ ]:
pred

In [ ]:
new['Traffic_Prediction_Next_Interval'] = pred[:, 0]
new['Traffic_Prediction_Next_Next_Interval'] = pred[:, 1]
new.head()

In [ ]:
for k, v in station_mapping.items():
    new.loc[new.Station == float(v), "Station_Name"] = k

In [ ]:
new[new.Station_Name == '1 AV'][["Station_Name", "DateTime", "Next_Interval_Value", "traffic_next_next_interval", "Traffic_Prediction_Next_Interval", "Traffic_Prediction_Next_Next_Interval"]]